# TODO

- fix classes according to kaggle rules: add unk and silence
- add hyperas optimization
- experiment with other kinds of data synthesis

In [4]:
import glob

import scipy.io.wavfile as wav
import scipy.signal
import pandas
from tqdm import tqdm, trange
from IPython.display import Audio
import pickle
import seaborn
from scipy.ndimage.interpolation import shift
# from soph import center_wave
seaborn.set_style("whitegrid")

%pylab inline

import random

Populating the interactive namespace from numpy and matplotlib


## Build an index dataframe

Read in the files and build labels and weights

- `raw_label`: 32 labels including those not in the target category.
- `label`: 12 labels including target labels, "silence" and "unknown"
- `raw_labels_i`: index for raw labels
- `labels_i`: index for labels
- `class_weight`: balanced class weights for examples

In [5]:
all_files = glob.glob("data/**/*.wav", recursive=True)
len(all_files)

64727

In [6]:
ex_df = pandas.DataFrame()
ex_df["fn"] = all_files
ex_df.head()

,fn
0,data/train/audio/right/988e2f9a_nohash_0.wav
1,data/train/audio/right/1eddce1d_nohash_3.wav
2,data/train/audio/right/93ec8b84_nohash_0.wav
3,data/train/audio/right/6272b231_nohash_1.wav
4,data/train/audio/right/439c84f4_nohash_1.wav


In [7]:
with open("data/train/validation_list.txt") as f:
    val_list = ["data/train/audio/"+fn.strip() for fn in f.readlines()]
with open("data/train/testing_list.txt") as f:
    test_list = ["data/train/audio/"+fn.strip() for fn in f.readlines()]

In [8]:
val_list[0]

'data/train/audio/bed/026290a7_nohash_0.wav'

In [9]:
ex_df["state"] = "train"
ex_df.state[ex_df.fn.str.contains("_background_noise_")]="bg"
ex_df.state[ex_df.fn.str.contains("test")]="submission"
ex_df.state[ex_df.fn.isin(val_list)]="val"
ex_df.state[ex_df.fn.isin(test_list)]="test"
ex_df.state.value_counts()

train    51088
test      6835
val       6798
bg           6
Name: state, dtype: int64

In [10]:
ex_df["raw_label"] = np.nan
label_bool = ex_df.state.isin(["test", "train", "val"])
ex_df.loc[label_bool,"raw_label"] = ex_df.loc[label_bool,"fn"].str.split('/').str[3]

val_df = pandas.DataFrame([{"fn":"silence", "state":"val", "raw_label":"silence"}]*260)
test_df = pandas.DataFrame([{"fn":"silence", "state":"test", "raw_label":"silence"}]*260)
train_df = pandas.DataFrame([{"fn":"silence", "state":"train", "raw_label":"silence"}]*1800)
ex_df = pandas.concat((ex_df,val_df,test_df,train_df))
label_bool = ex_df.state.isin(["test", "train", "val"])

ex_df.raw_label.value_counts()

stop       2380
seven      2377
yes        2377
zero       2376
up         2375
no         2375
two        2373
go         2372
four       2372
one        2370
six        2369
on         2367
right      2367
nine       2364
down       2359
five       2357
off        2357
three      2356
left       2353
eight      2352
silence    2320
house      1750
dog        1746
marvin     1746
wow        1745
happy      1742
sheila     1734
cat        1733
tree       1733
bird       1731
bed        1713
Name: raw_label, dtype: int64

In [11]:
ex_df["label"] = np.nan
target_classes = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence"]
target_bool = ex_df.raw_label.isin(target_classes)
ex_df.loc[target_bool,"label"] = ex_df.loc[target_bool,"raw_label"]
ex_df.loc[label_bool&(-target_bool),"label"] = "unknown"
ex_df.label.value_counts()

unknown    41039
stop        2380
yes         2377
no          2375
up          2375
go          2372
on          2367
right       2367
down        2359
off         2357
left        2353
silence     2320
Name: label, dtype: int64

In [12]:
counts = ex_df.label.value_counts()
cw = dict(zip(counts.index,counts))

In [13]:
num_ex = sum(label_bool)
num_labels = len(ex_df.label.unique()) -1
mean_n = num_ex/num_labels
counts = ex_df.label.value_counts()
cw = dict(zip(counts.index,counts))
for k,v in cw.items():
    cw[k] = mean_n/v
ex_df["label_weight"] = ex_df.label.map(cw)

In [14]:
label_list = target_classes + ["unknown"]
raw_label_list = ex_df.raw_label.unique()
raw_label_list = [lb for lb in raw_label_list if lb not in label_list]
raw_label_list.remove(np.nan)
raw_label_list = label_list + raw_label_list
raw_label_dict = dict(zip(raw_label_list, arange(len(raw_label_list))) )
ex_df["label_i"] = ex_df.label.map(raw_label_dict)
ex_df["raw_label_i"] = ex_df.raw_label.map(raw_label_dict)

In [15]:
ex_df.to_pickle("data/ex_df.pkl")

# Make figures

In [ ]:
with open("data/bg_list.txt") as f:
    bg_files = [fn.strip() for fn in f.readlines()]
bg_files
ex_df = pandas.concat([ex_df, pandas.DataFrame(
    [{"fn":fn, "state":"bg"} for fn in bg_files]
)])

In [ ]:
ex_df.state.value_counts()

In [ ]:
4

In [ ]:
ex_df.to_pickle("data/ex_df.pkl")

In [ ]:
ex_df = pandas.read_pickle("data/ex_df.pkl")

In [ ]:
len(ex_df.label.unique())

In [ ]:
len(ex_df.label_raw.unique())

# Everything below this is old code!

In [ ]:
train_files = train_files - set(validation_files) - set(testing_files)

print("{} total files: {} train, {} validation, {} test".format(
    total_n_files, len(train_files), len(validation_files), len(testing_files)))

In [ ]:
ex_df = pandas.DataFrame()
ex_df["fn"] = train_files + validation_files + testing_files
ex_df["state"] = ["train"] * len(train_files) + ["val"] * len(
    validation_files) + ["test"] * len(testing_files)
ex_df["cat"] = ex_df.fn.map(lambda x: x.split("/")[3])

ex_df.state[ex_df.cat == "_background_noise_"] = "bg"


# unk_cat = set(ex_df.cat.unique()) - set(target_cat)
# ex_df.cat[ex_df.cat.isin(unk_cat)] = "unknown"

val_df = pandas.DataFrame([{"fn":"silence", "state":"val", "cat":"silence"}]*260)
test_df = pandas.DataFrame([{"fn":"silence", "state":"test", "cat":"silence"}]*260)
train_df = pandas.DataFrame([{"fn":"silence", "state":"train", "cat":"silence"}]*1800)
ex_df = pandas.concat((ex_df,val_df,test_df,train_df))

In [ ]:
ex_df.describe()

In [ ]:
# build indices
unique_cat = ex_df.cat.unique()
cat_dict = dict(zip(unique_cat, range(len(unique_cat))))
with open("logs/cat_dict.pkl", "wb") as f:
    pickle.dump(cat_dict, f)
print(cat_dict)

In [ ]:
ex_df["cat_i"] = ex_df.cat.map(cat_dict)
ex_df.head()

In [ ]:
ex_df.to_pickle("data/ex_df.pkl")

In [ ]:
ex_df.state.value_counts()

In [ ]:
num_cat = len(ex_df.cat.unique())
print(num_cat)
ex_df[ex_df.state=="train"].cat.value_counts()

In [ ]:
def scaler(wave):
    return 2*(0.+ wave - wave.min())/(.0 + wave.max() - wave.min() + 1e-4) - 1

bg_waves = [scaler(wav.read(fn)[1]) for fn in ex_df[ex_df.state == "bg"].fn]

def center_wave(wav_fn, maxlen=16000, vol_range=.3):
    
    if wav_fn == "silence":
        bg_wave = random.choice(bg_waves)
        bg_len = len(bg_wave)-maxlen
        bg_start = np.random.randint(bg_len)
        wave = bg_wave[bg_start:bg_start+maxlen]

    else:
        wave = scaler(wav.read(wav_fn)[1])

        if wave.shape[0] < maxlen:
            left_pad = (maxlen - wave.shape[0]) // 2
            right_pad = maxlen - wave.shape[0] - left_pad
            wave = np.pad(
                wave, (left_pad, right_pad), 'constant', constant_values=0)
        
        if vol_range > 0:
            bg_wave = center_wave("silence", maxlen)
            bg_vol = vol_range * np.random.rand(1)
            wave_vol = 1 - bg_vol
            wave = wave_vol*wave + bg_vol * bg_wave
            wave = np.clip(wave, -1, 1)
            

    return wave


## wave2mfcc parameters:
## numcep should be equal to numfilt
## windows can overlap (ie winlen can be greater than winstep)

def wave2mfcc(wav_fn, vol_range=.3):

    rate = 16000
    
    wave = center_wave(wav_fn, vol_range=vol_range)

    
    mfcc_feat = mfcc(
        wave,
        rate,
        numcep=160,
        winlen=100 / 16000,
        winstep=100 / 16000,
        nfilt=160,
#         lowfreq=100,
#         highfreq=5000
    )

    return mfcc_feat

def wave2melspec(wav_fn):
    wave = center_wave(wav_fn)
    S = librosa.feature.melspectrogram(wave, sr=16000, n_mels=128)

    # Convert to log scale (dB). We'll use the peak power (max) as reference.
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S

def wave2spec(wav_fn):
    wave = center_wave(wav_fn)
    
    spec = np.log(np.abs(scipy.signal.stft(wave, fs=16000)[2])+1e-6)


    return spec

In [ ]:
len()

In [ ]:
for fn in tqdm(glob.glob("data/**/*.wav", recursive=True)):
    wav.write(fn, 16000,scaler(wav.read(fn)[1]))

In [ ]:
wave_out = wav.read(train_files[1])[1]
wave_out = scaler(wave_out)
wav.write("test.wav", 16000,wave_out)
wave2 = wav.read("test.wav")[1]

In [ ]:
all(wave2 == wave_out)

In [ ]:
out = center_wave(train_files[1], vol_range=.1)
Audio(data=out, rate=16000, embed=True)

In [ ]:
figsize(3,3)
plt.imshow(wave2mfcc(train_files[1]).T, aspect='auto');

In [ ]:
MFCC = True
if MFCC:
    x = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "train"].fn):

        #get the wave data w/o noise
#         mfcc_feat = wave2mfcc(fn, vol_range=0)

#         x.append(mfcc_feat[np.newaxis, :, :, np.newaxis])
        
        # get the wave data w/ noise
        mfcc_feat = wave2mfcc(fn, vol_range=.1)

        x.append(mfcc_feat[np.newaxis, :, :, np.newaxis])
        

    x = np.concatenate(x, axis=0)
    y = ex_df[ex_df.state == "train"].cat_i.values[:, np.newaxis]
    
    x_val = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "val"].fn):

        #get the wave data
        row = wave2mfcc(fn, vol_range=0)

        x_val.append(row[np.newaxis, :, :, np.newaxis])

    x_val = np.concatenate(x_val, axis=0)
    y_val = ex_df[ex_df.state == "val"].cat_i.values[:, np.newaxis]
    
    
    np.savez('data/xy-mfcc', x=x, y=y, x_val=x_val, y_val=y_val)
    
MS = False
if MS:
    x = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "train"].fn):

        #get the wave data
        mfcc_feat = wave2melspec(fn)

        x.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x = np.concatenate(x, axis=0)
    y = ex_df[ex_df.state == "train"].cat_i.values[:, np.newaxis]
    
    x_val = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "val"].fn):

        #get the wave data
        mfcc_feat = wave2melspec(fn)

        x_val.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x_val = np.concatenate(x_val, axis=0)
    y_val = ex_df[ex_df.state == "val"].cat_i.values[:, np.newaxis]
    
    
    np.savez('data/xy-ms', x=x, y=y, x_val=x_val, y_val=y_val)
    
SPEC = False
if SPEC:
    x = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "train"].fn):

        #get the wave data
        mfcc_feat = wave2spec(fn)

        x.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x = np.concatenate(x, axis=0)
    y = ex_df[ex_df.state == "train"].cat_i.values[:, np.newaxis]
    
    x_val = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "val"].fn):

        #get the wave data
        mfcc_feat = wave2spec(fn)

        x_val.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x_val = np.concatenate(x_val, axis=0)
    y_val = ex_df[ex_df.state == "val"].cat_i.values[:, np.newaxis]
    
    print(x.nbytes/1e9)
    np.savez('data/xy-spec', x=x, y=y, x_val=x_val, y_val=y_val)
    

In [ ]:
ex_df[ex_df.state=="val"].head()

In [ ]:
y_val[:5]

In [ ]:
Audio(center_wave("data/train/audio/bed/099d52ad_nohash_0.wav", vol_range=0), rate=10000)

In [ ]:
list(zip(np.unique(y),np.bincount(y.flat)))

In [ ]:
from sklearn.model_selection import KFold

In [ ]:


classes = np.arange(num_cat)

cw = utils.class_weight.compute_class_weight("balanced",classes.flat, y.flat)
cw = dict(zip(classes,cw))

In [ ]:
x.nbytes/1e9

In [ ]:
PROCESS_BG = False
if PROCESS_BG:
    x = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "train"].fn):

        #get the wave data
        mfcc_feat = wave2mfcc(fn, bg=True)

        x.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x = np.concatenate(x, axis=0)
    y = ex_df[ex_df.state == "train"].cat_i.values[:, np.newaxis]
    
    
    
    np.savez_compressed('data/xy_bg', x=x, y=y)

In [ ]:
PROCESS_BG = True
if PROCESS_BG:
    x_val = []

    # example loop
    for fn in tqdm(ex_df[ex_df.state == "val"].fn):

        #get the wave data
        mfcc_feat = wave2mfcc(fn, bg=True)

        x_val.append(mfcc_feat[np.newaxis, :, :, np.newaxis])

    x_val = np.concatenate(x_val, axis=0)
    y_val = ex_df[ex_df.state == "val"].cat_i.values[:, np.newaxis]
    np.savez_compressed('data/xy_val', x=x_val, y=y_val)

In [ ]:
def train_generator(batch_size=32, shuffle=True, state="train", maxlen = 16000):
    
    maxlen = 16000
    
    epoch_df = ex_df[ex_df.state == state]
    num_ex = len(epoch_df)
    indices = np.arange(num_ex)
    
    # epoch loop runs
    while True:
        
        # shuffle anew every epoch
        if shuffle:
            epoch_df = epoch_df.sample(frac=1)
        
        # batch loop
        for i in np.arange(0, num_ex, batch_size):
            
            x = []
            
            batch_df = epoch_df.iloc[i:i+batch_size:,:]
            
            # example loop
            for fn in batch_df.fn:
                                
                #get the processed file
                item = wave2spec(fn)
                
                x.append(item[np.newaxis,:,:,np.newaxis])
            
            x = np.concatenate(x,axis=0)
            y = batch_df.cat_i.values[:,np.newaxis]
            yield x, y

In [ ]:
x.shape[1:]

In [ ]:
dropout_prob = .4
init_stddev = 0.01

ff_layers = [
    keras.layers.InputLayer(input_shape=x.shape[1:]),
    keras.layers.Flatten(),
    keras.layers.Dropout(dropout_prob),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dropout(dropout_prob),

    # Classification
    keras.layers.Dense(
        num_cat,
        activation="softmax",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
]
ff_model = keras.Sequential(ff_layers)
ff_model.summary()

In [ ]:
x.shape[1:]

In [ ]:
dropout_prob = .4
init_stddev = 0.01

cnn_layers =[
    keras.layers.InputLayer(input_shape=x.shape[1:]),

    # Conv layer
    keras.layers.Conv2D(
        64,
        kernel_size=(8, 20),
        padding="same",
        activation="relu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.AvgPool2D(pool_size=(2, 2), padding="same"),
    keras.layers.Dropout(dropout_prob),
    
    # Conv layer
    keras.layers.Conv2D(
        64,
        kernel_size=(4, 10),
        padding="same",
        activation="relu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.Flatten(),
    keras.layers.Dropout(dropout_prob),


    
    # Class Layer 2
    keras.layers.Dense(
        num_cat,
        activation="softmax",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
]
cnn_layers = keras.Sequential(cnn_layers)
cnn_layers.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        monitor='val_loss',
        verbose=1,
        save_best_only=False,
        save_weights_only=False,
        mode='auto',
        period=1),
    keras.callbacks.EarlyStopping(patience=4, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.1, patience=2, verbose=1, min_lr=1e-6)
]

cnn_layers.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Nadam(lr=0.001),
    metrics=['accuracy'])

In [ ]:
cnn_layers.fit(
    x=np.concatenate((x,x_bg)),
    y=np.concatenate((y,y)),
    validation_data=(x_val, y_val),
    epochs=100,
    verbose=1,
    callbacks=callbacks,
    initial_epoch=0)

# cnn_layers.fit_generator(
#     generator=train_generator(
#         batch_size=batch_size, shuffle=True, state="train", maxlen=16000),
#     steps_per_epoch=num_steps,
#     epochs=100,
#     verbose=1,
#     callbacks=callbacks,
#     validation_data=train_generator(
#         batch_size=batch_size, shuffle=True, state="val", maxlen=16000),
#     validation_steps=val_steps,
#     shuffle=False,
#     initial_epoch=0,
#     max_queue_size=100)